## Benchmark model

In this notebook a Simple Moving Average model will be created in order to have a benchmark for my DeepAR based model (to be tuned in the next notebook).

### Hyperparameters

DeepAR is the model of choice for this project.
This model expects input data to be already test-train splitted.
A big part of the model design has to be done looking close at data.
More specifically, defining these two hyperparameters about the data:
* Context length
* Prediction length

### Prediction length

This is the length of the time series future predictions in days. It will be conveniently set to 5 days (exactly a week of trading hours) because a shorter interval would be of little significance.
A longer interval could be interesting from an application point of view, but it can be challenging in terms of model performances.

### Context length

Context length can be either:
* designed on patterns or seasonality observed in the data, if any is present;
* chosen as a fixed value. This will be my choice, and it will be the same as the moving average window, in order to have a good reference metrics, applicable to both this model and the benchmark model.

To explore this second option, we will refer to what we've found during the EDA stage.

In [138]:
prediction_length = [10, 20, 50]
context_length = [10, 20, 50]

### Train, test and validation split

Time series will be all trimmed from `data_start` in `1.ExploratoryDataAnalysis.ipy`, according to DeepAR documentation, train time series should be set as the size of the entire time series less the prediction length.
Validation length will exactly equal to prediction length.

In [139]:
from data.data_utils import train_test_valid_split

#### IBM Stock train, test and validation split

In [140]:
df_ibm = df_ibm.loc[data_start:].copy()

In [141]:
df_ibm_train, df_ibm_test, df_ibm_valid = train_test_valid_split(df_ibm, prediction_length=prediction_length[1])
print(len(df_ibm), len(df_ibm_train), len(df_ibm_test), len(df_ibm_valid))

4194 4154 4174 20


In [142]:
df_ibm_train.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,54.979382,84.889999,85.349998,84.449997,84.750000,4704500,0.18,0.0,54.397126,54.958798,55.913001,55.525647,53.268605,56.500281,53.417314,59.046566,52.779437
2004-08-20,55.212502,85.250000,85.250000,84.519997,84.519997,4501400,0.18,0.0,54.511759,54.977556,55.852774,55.720874,53.302644,56.521944,53.433168,58.920109,52.785438
2004-08-23,54.823910,84.650002,85.449997,84.650002,85.230003,4260600,0.18,0.0,54.582999,54.969128,55.779992,55.771068,53.394930,56.515013,53.423244,58.766151,52.793832
2004-08-24,54.862782,84.709999,85.150002,84.349998,85.000000,2710400,0.18,0.0,54.564866,54.937116,55.713028,55.726906,53.402826,56.462835,53.411398,58.625949,52.800108
2004-08-25,55.095955,85.070000,85.269997,84.550003,85.000000,4405600,0.18,0.0,54.654243,54.917732,55.644653,55.829713,53.478772,56.423940,53.411524,58.447571,52.841736


In [143]:
df_ibm_test.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,54.979382,84.889999,85.349998,84.449997,84.750000,4704500,0.18,0.0,54.397126,54.958798,55.913001,55.525647,53.268605,56.500281,53.417314,59.046566,52.779437
2004-08-20,55.212502,85.250000,85.250000,84.519997,84.519997,4501400,0.18,0.0,54.511759,54.977556,55.852774,55.720874,53.302644,56.521944,53.433168,58.920109,52.785438
2004-08-23,54.823910,84.650002,85.449997,84.650002,85.230003,4260600,0.18,0.0,54.582999,54.969128,55.779992,55.771068,53.394930,56.515013,53.423244,58.766151,52.793832
2004-08-24,54.862782,84.709999,85.150002,84.349998,85.000000,2710400,0.18,0.0,54.564866,54.937116,55.713028,55.726906,53.402826,56.462835,53.411398,58.625949,52.800108
2004-08-25,55.095955,85.070000,85.269997,84.550003,85.000000,4405600,0.18,0.0,54.654243,54.917732,55.644653,55.829713,53.478772,56.423940,53.411524,58.447571,52.841736


In [144]:
df_ibm_valid.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2021-03-19,128.899994,128.899994,130.440002,128.529999,130.020004,9830600,NaN,NaN,127.642000,124.448500,123.117711,131.348088,123.935912,131.734582,117.162418,131.037765,115.197658
2021-03-22,130.550003,130.550003,130.720001,127.889999,128.500000,4164900,NaN,NaN,128.216000,124.933000,123.182930,131.746530,124.685471,132.497730,117.368270,131.294574,115.071287
2021-03-23,130.460007,130.460007,131.559998,129.800003,130.440002,4356400,NaN,NaN,128.844001,125.420501,123.255428,131.233594,126.454408,133.095208,117.745794,131.562737,114.948119
2021-03-24,130.619995,130.619995,132.110001,130.570007,130.949997,4005000,NaN,NaN,129.119000,125.791000,123.330139,131.639766,126.598234,133.727371,117.854630,131.835490,114.824787
2021-03-25,133.070007,133.070007,133.240005,129.770004,130.330002,5554000,NaN,NaN,129.712001,126.321001,123.441415,132.872493,126.551509,134.725514,117.916487,132.307696,114.575135


#### Apple Inc. Stock train test split

In [145]:
df_aapl = df_aapl.loc[data_start:].copy()

In [146]:
df_aapl_train, df_aapl_test, df_aapl_valid = train_test_valid_split(df_aapl, prediction_length=prediction_length[1])
print(len(df_aapl), len(df_aapl_train), len(df_aapl_test), len(df_aapl_valid))

4194 4154 4174 20


In [147]:
df_aapl_train.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,0.472366,0.548393,0.568929,0.542143,0.562679,388920000.0,0.0,2.0,0.473627,0.481141,0.484422,0.491197,0.456057,0.504857,0.457425,0.519835,0.449008
2004-08-20,0.473750,0.550000,0.553393,0.544464,0.548393,316780800.0,0.0,2.0,0.475196,0.481218,0.484606,0.488983,0.461409,0.504821,0.457614,0.519691,0.449521
2004-08-23,0.478057,0.555000,0.558393,0.546429,0.551071,254660000.0,0.0,2.0,0.476395,0.481079,0.484711,0.488652,0.464139,0.504725,0.457433,0.519683,0.449738
2004-08-24,0.491439,0.570536,0.570536,0.556964,0.558214,374136000.0,0.0,2.0,0.477057,0.480710,0.485274,0.491797,0.462317,0.503401,0.458019,0.519742,0.450806
2004-08-25,0.508359,0.590179,0.591964,0.566607,0.569107,505618400.0,0.0,2.0,0.480195,0.481310,0.486000,0.504872,0.455517,0.506265,0.456355,0.520858,0.451141


In [148]:
df_aapl_test.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,0.472366,0.548393,0.568929,0.542143,0.562679,388920000.0,0.0,2.0,0.473627,0.481141,0.484422,0.491197,0.456057,0.504857,0.457425,0.519835,0.449008
2004-08-20,0.473750,0.550000,0.553393,0.544464,0.548393,316780800.0,0.0,2.0,0.475196,0.481218,0.484606,0.488983,0.461409,0.504821,0.457614,0.519691,0.449521
2004-08-23,0.478057,0.555000,0.558393,0.546429,0.551071,254660000.0,0.0,2.0,0.476395,0.481079,0.484711,0.488652,0.464139,0.504725,0.457433,0.519683,0.449738
2004-08-24,0.491439,0.570536,0.570536,0.556964,0.558214,374136000.0,0.0,2.0,0.477057,0.480710,0.485274,0.491797,0.462317,0.503401,0.458019,0.519742,0.450806
2004-08-25,0.508359,0.590179,0.591964,0.566607,0.569107,505618400.0,0.0,2.0,0.480195,0.481310,0.486000,0.504872,0.455517,0.506265,0.456355,0.520858,0.451141


In [149]:
df_aapl_valid.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2021-03-19,119.989998,119.989998,121.430000,119.680000,119.900002,185023200.0,NaN,NaN,121.525999,122.562000,129.351190,126.939457,116.112541,128.230984,116.893015,142.866324,115.836056
2021-03-22,123.389999,123.389999,123.870003,120.260002,120.330002,111912300.0,NaN,NaN,122.228999,122.431499,129.204497,126.326820,118.131179,127.883253,116.979746,142.817649,115.591345
2021-03-23,122.540001,122.540001,124.239998,122.139999,123.330002,95467100.0,NaN,NaN,122.374000,122.265499,129.018238,126.394583,118.353417,127.474474,117.056525,142.737912,115.298564
2021-03-24,120.089996,120.089996,122.900002,120.070000,122.820000,88530500.0,NaN,NaN,122.384999,122.002499,128.844287,126.376974,118.393025,127.085371,116.919628,142.794514,114.894060
2021-03-25,120.589996,120.589996,121.660004,119.000000,119.540001,98844700.0,NaN,NaN,122.247999,121.982499,128.683930,126.395791,118.100207,127.085255,116.879744,142.828230,114.539631


#### Amazon Stock train test split

In [150]:
df_amzn = df_amzn.loc[data_start:].copy()

In [151]:
df_amzn_train, df_amzn_test, df_amzn_valid = train_test_valid_split(df_amzn, prediction_length=prediction_length[1])
print(len(df_amzn), len(df_amzn_train), len(df_amzn_test), len(df_amzn_valid))

4194 4154 4174 20


In [152]:
df_amzn_train.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,38.630001,38.630001,40.480000,36.349998,40.259998,12696100.0,NaN,NaN,37.106001,37.6520,44.9828,39.862539,34.349462,40.417963,34.886038,57.693587,32.272014
2004-08-20,39.509998,39.509998,39.910000,38.110001,38.459999,6790800.0,NaN,NaN,37.508000,37.6285,44.7682,40.386916,34.629084,40.318115,34.938885,57.479021,32.057379
2004-08-23,39.450001,39.450001,40.000000,39.110001,39.889999,5532600.0,NaN,NaN,37.921000,37.6620,44.5584,40.581550,35.260450,40.427620,34.896380,57.267085,31.849715
2004-08-24,39.049999,39.049999,39.930000,38.320000,39.720001,7640400.0,NaN,NaN,38.116000,37.6590,44.3544,40.794891,35.437110,40.418129,34.899871,57.083144,31.625656
2004-08-25,40.299999,40.299999,40.490002,38.160000,39.060001,7254800.0,NaN,NaN,38.490000,37.7755,44.1582,41.246567,35.733433,40.776113,34.774887,56.827117,31.489283


In [153]:
df_amzn_test.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,38.630001,38.630001,40.480000,36.349998,40.259998,12696100.0,NaN,NaN,37.106001,37.6520,44.9828,39.862539,34.349462,40.417963,34.886038,57.693587,32.272014
2004-08-20,39.509998,39.509998,39.910000,38.110001,38.459999,6790800.0,NaN,NaN,37.508000,37.6285,44.7682,40.386916,34.629084,40.318115,34.938885,57.479021,32.057379
2004-08-23,39.450001,39.450001,40.000000,39.110001,39.889999,5532600.0,NaN,NaN,37.921000,37.6620,44.5584,40.581550,35.260450,40.427620,34.896380,57.267085,31.849715
2004-08-24,39.049999,39.049999,39.930000,38.320000,39.720001,7640400.0,NaN,NaN,38.116000,37.6590,44.3544,40.794891,35.437110,40.418129,34.899871,57.083144,31.625656
2004-08-25,40.299999,40.299999,40.490002,38.160000,39.060001,7254800.0,NaN,NaN,38.490000,37.7755,44.1582,41.246567,35.733433,40.776113,34.774887,56.827117,31.489283


In [154]:
df_amzn_valid.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2021-03-19,3074.959961,3074.959961,3077.290039,3016.629883,3029.229980,4616400.0,NaN,NaN,3068.773999,3079.814990,3183.646777,3170.161681,2967.386317,3210.955835,2948.674145,3410.511827,2956.781728
2021-03-22,3110.870117,3110.870117,3126.580078,3060.050049,3067.850098,2902200.0,NaN,NaN,3084.666016,3076.321497,3182.620981,3146.946573,3022.385458,3199.630779,2953.012214,3410.344783,2954.897180
2021-03-23,3137.500000,3137.500000,3182.000000,3120.850098,3127.000000,3817300.0,NaN,NaN,3092.131006,3073.471497,3181.716982,3160.397437,3023.864575,3187.570993,2959.372000,3409.798086,2953.635879
2021-03-24,3087.070068,3087.070068,3160.310059,3085.149902,3151.040039,2959000.0,NaN,NaN,3095.074023,3069.848499,3181.174185,3159.140136,3031.007911,3176.821323,2962.875674,3410.038860,2952.309509
2021-03-25,3046.260010,3046.260010,3109.780029,3037.139893,3072.989990,3563500.0,NaN,NaN,3088.341016,3069.303503,3179.682783,3157.692550,3018.989481,3176.658895,2961.948112,3411.110022,2948.255544


#### Alphabet Inc. Stock train test split

In [155]:
df_googl = df_googl.loc[data_start:].copy()

In [156]:
df_googl_train, df_googl_test, df_googl_valid = train_test_valid_split(df_googl, prediction_length=prediction_length[1])
print(len(df_googl), len(df_googl_train), len(df_googl_test), len(df_googl_valid))

4194 4154 4174 20


In [157]:
df_googl_train.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,50.220219,50.220219,52.082081,48.028027,50.050049,44659096.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-20,54.209209,54.209209,54.594597,50.300301,50.555557,22834343.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-23,54.754753,54.754753,56.796799,54.579578,55.430431,18256126.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-24,52.487488,52.487488,55.855858,51.836838,55.675674,15247337.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-25,53.053055,53.053055,54.054054,51.991993,52.532532,9188602.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381


In [158]:
df_googl_test.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,50.220219,50.220219,52.082081,48.028027,50.050049,44659096.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-20,54.209209,54.209209,54.594597,50.300301,50.555557,22834343.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-23,54.754753,54.754753,56.796799,54.579578,55.430431,18256126.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-24,52.487488,52.487488,55.855858,51.836838,55.675674,15247337.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-25,53.053055,53.053055,54.054054,51.991993,52.532532,9188602.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381


In [159]:
df_googl_valid.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2021-03-19,2026.959961,2026.959961,2037.040039,2002.930054,2029.729980,2302700.0,NaN,NaN,2050.343970,2050.801996,1977.966794,2110.719150,1989.968790,2108.769815,1992.834177,2230.033286,1725.900303
2021-03-22,2030.689941,2030.689941,2048.340088,2014.000000,2027.630005,1676800.0,NaN,NaN,2052.662964,2049.623492,1983.093794,2107.225927,1998.100001,2108.249933,1990.997052,2228.598041,1737.589547
2021-03-23,2041.329956,2041.329956,2064.110107,2028.000000,2039.949951,1425500.0,NaN,NaN,2052.759961,2048.683984,1987.963794,2107.229087,1998.290835,2107.204305,1990.163664,2228.069047,1747.858541
2021-03-24,2032.530029,2032.530029,2067.600098,2029.000000,2051.760010,1260900.0,NaN,NaN,2052.393970,2046.119983,1993.488594,2107.404425,1997.383515,2102.618906,1989.621059,2224.370890,1762.606297
2021-03-25,2032.459961,2032.459961,2045.640015,1996.089966,2029.050049,1780800.0,NaN,NaN,2045.585962,2046.945483,1999.389192,2089.931418,2001.240505,2102.053695,1991.837272,2218.332520,1780.445864


### Metrics computation

Computing metrics on the benchmark model will give me a good reference to evaluate the deep learning model after training.

As can be presumed and seen from data prints, first `n` values of validation data are `NaN` so, it will be better just to exclude the first `n-1` values from the validation set to avoid misleading values to be catch in metrics evaluation. 

In [160]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score

#### IBM stock

In [161]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [162]:
#ibm_ma_mse_loss = mean_squared_error(df_ibm_valid.iloc[n-1:]['Adj Close'], df_ibm_valid.iloc[n-1:][ma_str])

In [163]:
#print(ibm_ma_mse_loss)

In [164]:
n = 20
ma_str = str(n)+'_ac_ma'

Mean Absolute Error

In [165]:
ibm_ma_mae_loss = mean_absolute_error(df_ibm_valid.iloc[:]['Adj Close'], df_ibm_valid[:][ma_str])

In [166]:
print(ibm_ma_mae_loss)

4.033148365020752


Root Mean Squared Error

In [167]:
ibm_ma_mse_loss = mean_squared_error(df_ibm_valid.iloc[:]['Adj Close'], df_ibm_valid[:][ma_str], squared=False)

In [168]:
print(ibm_ma_mse_loss)

4.673916323212443


Mean Absolute Percentage Error

In [169]:
ibm_ma_map_loss = mean_absolute_percentage_error(df_ibm_valid.iloc[:]['Adj Close'], df_ibm_valid[:][ma_str])

In [170]:
print(ibm_ma_map_loss)

0.030186435471614002


R<sup>2</sup> score

In [171]:
ibm_ma_r2_score = r2_score(df_ibm_valid.iloc[:]['Adj Close'], df_ibm_valid[:][ma_str])

In [172]:
print(ibm_ma_r2_score)

-4.021170554272494


In [173]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [174]:
#ibm_ma_mse_loss = mean_squared_error(df_ibm_valid.iloc[n-1:]['Adj Close'], df_ibm_valid[n-1:][ma_str])

In [175]:
#print(ibm_ma_mse_loss)

#### Apple Inc. stock

In [176]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [177]:
#aapl_ma_mse_loss = mean_squared_error(df_aapl_valid.iloc[n-1:]['Adj Close'], df_aapl_valid.iloc[n-1:][ma_str])

In [178]:
#print(aapl_ma_mse_loss)

In [179]:
n = 20
ma_str = str(n)+'_ac_ma'

Mean Absolute Error

In [180]:
aapl_ma_mae_loss = mean_absolute_error(df_aapl_valid.iloc[:]['Adj Close'], df_aapl_valid[:][ma_str])

In [181]:
print(aapl_ma_mae_loss)

4.125100803375243


##### Root Mean Squared Error

In [182]:
aapl_ma_rmse_loss = mean_squared_error(df_aapl_valid.iloc[:]['Adj Close'], df_aapl_valid[:][ma_str], squared=False)

In [183]:
print(aapl_ma_rmse_loss)

5.273220267048253


##### Mean Absolute Percentage Error

In [184]:
aapl_ma_map_loss = mean_absolute_percentage_error(df_aapl_valid.iloc[:]['Adj Close'], df_aapl_valid[:][ma_str])

In [185]:
print(aapl_ma_map_loss)

0.03171177121153868


R<sup>2</sup> score

In [186]:
aapl_ma_r2_score = r2_score(df_aapl_valid.iloc[:]['Adj Close'], df_aapl_valid[:][ma_str])

In [187]:
print(aapl_ma_r2_score)

0.0224850740131185


In [188]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [189]:
#aapl_ma_mse_loss = mean_squared_error(df_aapl_valid.iloc[n-1:]['Adj Close'], df_aapl_valid[n-1:][ma_str])

In [190]:
#print(aapl_ma_mse_loss)

#### Amazon.com stock

In [191]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [192]:
#amzn_ma_mse_loss = mean_squared_error(df_amzn_valid.iloc[n-1:]['Adj Close'], df_amzn_valid.iloc[n-1:][ma_str])

In [193]:
#print(amzn_ma_mse_loss)

In [194]:
n = 20
ma_str = str(n)+'_ac_ma'

Mean Absolute Error

In [195]:
amzn_ma_mae_loss = mean_absolute_error(df_amzn_valid.iloc[:]['Adj Close'], df_amzn_valid[:][ma_str])

In [196]:
print(amzn_ma_mae_loss)

106.48609680175782


Root Mean Squared Error

In [197]:
amzn_ma_rmse_loss = mean_squared_error(df_amzn_valid.iloc[:]['Adj Close'], df_amzn_valid[:][ma_str], squared=False)

In [198]:
print(amzn_ma_rmse_loss)

135.42556190249684


##### Mean Absolute Percentage Error

In [199]:
amzn_ma_map_loss = mean_absolute_percentage_error(df_amzn_valid.iloc[:]['Adj Close'], df_amzn_valid[:][ma_str])

In [200]:
print(amzn_ma_map_loss)

0.03220080058758169


R<sup>2</sup> score

In [201]:
amzn_ma_r2_score = r2_score(df_amzn_valid.iloc[:]['Adj Close'], df_amzn_valid[:][ma_str])

In [202]:
print(amzn_ma_r2_score)

-0.07264587834183422


In [203]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [204]:
#amzn_ma_mse_loss = mean_squared_error(df_amzn_valid.iloc[n-1:]['Adj Close'], df_amzn_valid[n-1:][ma_str])

In [205]:
#print(amzn_ma_mse_loss)

#### Alphabet Inc. stock

In [206]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [207]:
#googl_ma_mse_loss = mean_squared_error(df_googl_valid.iloc[n-1:]['Adj Close'], df_googl_valid.iloc[n-1:][ma_str])

In [208]:
#print(googl_ma_mse_loss)

In [209]:
n = 20
ma_str = str(n)+'_ac_ma'

Mean Absolute Error

In [210]:
googl_ma_mae_loss = mean_absolute_error(df_googl_valid.iloc[:]['Adj Close'], df_googl_valid[:][ma_str])

In [211]:
print(googl_ma_mae_loss)

83.27505523681639


Root Mean Squared Error

In [212]:
googl_ma_rmse_loss = mean_squared_error(df_googl_valid.iloc[:]['Adj Close'], df_googl_valid[:][ma_str], squared=False)

In [213]:
print(googl_ma_rmse_loss)

106.54646100406165


##### Mean Absolute Percentage Error

In [214]:
googl_ma_map_loss = mean_absolute_percentage_error(df_googl_valid.iloc[:]['Adj Close'], df_googl_valid[:][ma_str])

In [215]:
print(googl_ma_map_loss)

0.037390411229982795


R<sup>2</sup> score

In [216]:
googl_ma_r2_score = r2_score(df_googl_valid.iloc[:]['Adj Close'], df_googl_valid[:][ma_str])

In [217]:
print(googl_ma_r2_score)

-0.03602134087429065


In [218]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [219]:
#googl_ma_mse_loss = mean_squared_error(df_googl_valid.iloc[n-1:]['Adj Close'], df_googl_valid[n-1:][ma_str])

In [220]:
#print(googl_ma_mse_loss)

Volatility

In [221]:
print(volatility(df_ibm_valid['Adj Close'], n))

0.22898302675871207


In [222]:
print(volatility(df_aapl_valid['Adj Close'], n))

1.4971827913918478


In [223]:
print(volatility(df_amzn_valid['Adj Close'], n))

899.8939315887699


In [224]:
print(volatility(df_googl_valid['Adj Close'], n))

576.7077073259011


As expected, loss augments as we observe moving average on larger windows.
Also, we can observe that Amazon.com and Alphabet Inc. have greater losses, that also corresponds to higher volatility.

Now I'll initialize an array of moving average values on validation data, to be used in future model comparison:

In [225]:
ser_valid_ma = [df_ibm_valid[:][ma_str], df_aapl_valid[:][ma_str], df_amzn_valid[:][ma_str], df_googl_valid[:][ma_str]]